<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Activity" data-toc-modified-id="Activity-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Activity</a></span></li><li><span><a href="#Setting-up-data" data-toc-modified-id="Setting-up-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Setting up data</a></span></li><li><span><a href="#Over-time" data-toc-modified-id="Over-time-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Over time</a></span></li><li><span><a href="#Over-time" data-toc-modified-id="Over-time-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Over time</a></span></li><li><span><a href="#Plotting" data-toc-modified-id="Plotting-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Plotting</a></span></li></ul></div>

# Activity

https://towardsdatascience.com/spread-of-covid-19-with-interactive-data-visualization-521ac471226e?gi=e6b7e5175fad

    data structure: 0 (healthy), 1 (newly infected), 2 (infectious), 3 (Dead), 9: (immune)
    n x n = tissue (with values 0:3)
    plot with heatmap
    gent-based modeling with time-steps
    ~35min

In [1]:
import matplotlib.pyplot
import pandas as pd
import random
import plotly as py
import plotly.graph_objs as go

# Setting up data

healthy, infectious, newly infected

In [85]:
n = 50

#n_starting_infected = 10
n_starting_infected = random.randint(0, 5)
n_starting_immune = random.randint(0, 5)

current = pd.DataFrame(0, index = range(n), columns = range(n))
points = set()
while len(points) < n_starting_infected:
    points.add((random.randint(0, n-1), random.randint(0, n-1)))

In [86]:
points

{(5, 46), (28, 47)}

In [87]:
INFECT_PROB = 0.5
for point in points:
    current.iloc[point[0], point[1]] = 1

def one_time_step(current, infect_prob):
    
    updated = current.copy()
    for row in range(n):
        for col in range(n):

            if current.iloc[row, col] == 0:
                if row != n-1:
                    if current.iloc[row+1,col] == 2:
                        if random.random() < infect_prob:
                            updated.iloc[row,col] = 1

                if col != n-1:       
                    if current.iloc[row,col+1] == 2:
                        if random.random() < infect_prob:
                            updated.iloc[row,col] = 1

                if row != 0:
                    if current.iloc[row-1,col] == 2:
                        if random.random() < infect_prob:
                            updated.iloc[row,col] = 1

                if col != 0:
                    if current.iloc[row,col-1] == 2:
                        if random.random() < infect_prob:
                            updated.iloc[row,col] = 1

            if current.iloc[row, col] == 1:
                updated.iloc[row, col] = 2

            if current.iloc[row, col] == 2:
                updated.iloc[row, col] = 3

            if current.iloc[row, col] == 3:
                updated.iloc[row, col] = 3 
                
    return(updated)
            

def t_time_step(t, current, infect_prob):
    
    out = []
    
    out.append(current.values)
    
    updated = current.copy()
    for step in range(t):
        updated = one_time_step(updated, infect_prob)
        out.append(updated.values)
        
    return(out)
        
def heatmap():
    pass
        
# if there is a any cell that is infectious, then there is some probability that it becomes infeccted (50%)

In [88]:
(current == 1).sum().sum()

2

# Over time

In [89]:
res = t_time_step(50, current, infect_prob = INFECT_PROB)

In [90]:
heatmaps=res

In [91]:
fig = go.Figure(data=go.Heatmap(z=heatmaps[0]),
               frames=[go.Frame(data=go.Heatmap(z=heatmaps[i])) for i in range(len(res))])
fig.update_layout(
    updatemenus=[
        dict(type="buttons", visible=True,
        buttons=[dict(label="Play", method="animate", args=[None])]
            )])
fig.show()

Add immune system idea 
- First stage infected cells are killed before infecting others with some probability
- Autophagy turns 1s and 2s into 3s
- Add an immune cell state that can potentially move (randomly)
- Have infectious process separate from immune cells

Probability associated with number of starting infected cells

healthy, infectious, newly infected

In [118]:
n = 10

#n_starting_infected = 10
n_starting_infected = random.randint(0, 5)
n_starting_immune = random.randint(0, 10)
current = pd.DataFrame(0, index = range(n), columns = range(n))

In [119]:
# Add infected cells
infected_points = set()
while len(infected_points) < n_starting_infected:
    infected_points.add((random.randint(0, n-1), random.randint(0, n-1)))
for point in infected_points:
    current.iloc[point[0], point[1]] = 1

In [120]:
# Add immune cells
immune_points = set()
while len(immune_points) < n_starting_immune:
    immune_points.add((random.randint(0, n-1), random.randint(0, n-1)))
for point in immune_points:
    current.iloc[point[0], point[1]] = 9

In [121]:
INFECT_PROB = 0.8

def one_time_step(current, infect_prob):
    
    updated = current.copy()
    for row in range(n):
        for col in range(n):

            if current.iloc[row, col] == 0:
                if row != n-1:
                    if current.iloc[row+1,col] == 2:
                        if random.random() < infect_prob:
                            updated.iloc[row,col] = 1

                if col != n-1:       
                    if current.iloc[row,col+1] == 2:
                        if random.random() < infect_prob:
                            updated.iloc[row,col] = 1

                if row != 0:
                    if current.iloc[row-1,col] == 2:
                        if random.random() < infect_prob:
                            updated.iloc[row,col] = 1

                if col != 0:
                    if current.iloc[row,col-1] == 2:
                        if random.random() < infect_prob:
                            updated.iloc[row,col] = 1

            if current.iloc[row, col] == 1:
                updated.iloc[row, col] = 2

            if current.iloc[row, col] == 2:
                #probability
                updated.iloc[row, col] = 3

            if current.iloc[row, col] == 3:
                updated.iloc[row, col] = 3 
                
    return(updated)  

In [122]:
def immune_time_step(current):
    updated = current.copy()
    for row in range(n):
        for col in range(n):
            if current.iloc[row, col] == 9:
                if row != n-1:
                    if current.iloc[row+1,col] in [1,2]:
                        updated.iloc[row+1,col] = 3

                if col != n-1:       
                    if current.iloc[row,col+1]in [1,2]:
                        updated.iloc[row,col+1] = 3

                if row != 0:
                    if current.iloc[row-1,col] in [1,2]:
                        updated.iloc[row-1,col] = 3

                if col != 0:
                    if current.iloc[row,col-1] in [1,2]:
                        updated.iloc[row,col-1] = 3
    return(updated)

In [123]:
def t_time_step(t, current, infect_prob):
    
    out = []
    
    out.append(current.values)
    
    updated = current.copy()
    for step in range(t):
        updated = immune_time_step(updated)
        updated = one_time_step(updated, infect_prob)
        out.append(updated.values)
        
    return(out)
        
def heatmap():
    pass
        
# if there is a any cell that is infectious, then there is some probability that it becomes infeccted (50%)

In [124]:
(current == 1).sum().sum()

1

# Over time

In [125]:
res = t_time_step(100, current, infect_prob = INFECT_PROB)

In [126]:
heatmaps=res

In [127]:
fig = go.Figure(data=go.Heatmap(z=heatmaps[0]),
               frames=[go.Frame(data=go.Heatmap(z=heatmaps[i])) for i in range(len(res))])
fig.update_layout(
    updatemenus=[
        dict(type="buttons", visible=True,
        buttons=[dict(label="Play", method="animate", args=[None])]
            )])
fig.show()

# Plotting

In [8]:
colorscale = [
    [0, 'blue'],
    [1, 'red'],
    [2, 'green'],
    [3, 'yellow'],
]